#databricks notebook: GOLD - FireRisk Diário (INPE)

In [0]:
# Databricks notebook: GOLD - FireRisk Diário (INPE)
# =========================================================

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, avg, min as spark_min, max as spark_max, count, current_timestamp 
)

spark = SparkSession.builder.getOrCreate()

In [0]:
# =========================================================
# 🔹 Parâmetros
# =========================================================

dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema_silver", "")
dbutils.widgets.text("table_silver", "")
dbutils.widgets.text("schema_gold", "")
dbutils.widgets.text("table_gold", "")
dbutils.widgets.text("data_ref_carga", "")

catalog         = dbutils.widgets.get("catalog")
schema_silver   = dbutils.widgets.get("schema_silver")
table_silver    = dbutils.widgets.get("table_silver")
schema_gold     = dbutils.widgets.get("schema_gold")
table_gold      = dbutils.widgets.get("table_gold")
data_ref_carga  = dbutils.widgets.get("data_ref_carga")

if not data_ref_carga:
    raise ValueError("❌ Parâmetro 'data_ref_carga' não informado")

print("=====================================")
print(f"📦 Silver origem : {catalog}.{schema_silver}.{table_silver}")
print(f"💾 Gold destino  : {catalog}.{schema_gold}.{table_gold}")
print(f"📅 Data solicitada: {data_ref_carga if data_ref_carga else 'NÃO INFORMADA'}")
print("=====================================")

In [0]:
# =========================================================
# 🔹 Localizar última partição válida na Silver
# =========================================================

silver_full = spark.table(f"{catalog}.{schema_silver}.{table_silver}") \
                    .select("data_ref_carga") \
                    .distinct()

datas_disponiveis = [r["data_ref_carga"] for r in silver_full.collect()]

if not datas_disponiveis:
    raise RuntimeError("❌ Nenhuma partição disponível na Silver.")

# Ordenar datas disponíveis
datas_ordenadas = sorted(datas_disponiveis)

# Caso usuário informe a data
if data_ref_carga:
    # Buscar a última ≤ data informada
    datas_validas = [d for d in datas_ordenadas if d <= data_ref_carga]

    if not datas_validas:
        raise RuntimeError(
            f"❌ Nenhuma partição encontrada ≤ {data_ref_carga} na Silver."
        )

    data_usada = datas_validas[-1]  # última menor ou igual
else:
    # Nenhuma data informada → usa a última partição existente
    data_usada = datas_ordenadas[-1]

print(f"📌 Data utilizada para GOLD: {data_usada}")

In [0]:
# =========================================================
# Leitura da Silver
# =========================================================
df_silver = spark.table(f"{catalog}.{schema_silver}.{table_silver}") \
                 .filter(col("data_ref_carga") == data_usada)

total_linhas = df_silver.count()
print(f"🔸 Registros Silver lidos: {total_linhas}")

if total_linhas == 0:
    raise RuntimeError(f"❌ Silver vazia para data {data_usada}. Encerrando job.")

In [0]:
# =========================================================
# 🔹 Agregação GOLD — Resumo Diário Brasil
# =========================================================
df_gold = (
    df_silver
    .groupBy("data_ref_carga")
    .agg(
        avg("rf").alias("rf_medio"),
        spark_min("rf").alias("rf_min"),
        spark_max("rf").alias("rf_max"),
        count("*").alias("qtde_pontos_grade")
    )
    .withColumn("processado_em", current_timestamp())
)

print("🔸 Gold gerada com sucesso")

In [0]:
# =========================================================
# 🔹 Escrita Gold (Delta)
# =========================================================
(
    df_gold.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"data_ref_carga = '{data_ref_carga}'")
    .saveAsTable(f"{catalog}.{schema_gold}.{table_gold}")
)

print(f"💾 Gold salva em: {catalog}.{schema_gold}.{table_gold}")

In [0]:
# =========================================================
# 🔹 Comentários da GOLD (Unity Catalog)
# =========================================================

# Comentário da tabela
spark.sql(f"""
ALTER TABLE {catalog}.{schema_gold}.{table_gold}
SET TBLPROPERTIES (
  'comment' = 'Tabela GOLD - Resumo diário do risco de fogo INPE FireRisk 2.2 (média, mínimo, máximo, quantidade de pontos).'
)
""")

# Comentários das colunas
spark.sql(f"""
ALTER TABLE {catalog}.{schema_gold}.{table_gold}
ALTER COLUMN data_ref_carga COMMENT 'Data de referência AAAAMMDD'
""")

spark.sql(f"""
ALTER TABLE {catalog}.{schema_gold}.{table_gold}
ALTER COLUMN rf_medio COMMENT 'Valor médio do risco de fogo no dia'
""")

spark.sql(f"""
ALTER TABLE {catalog}.{schema_gold}.{table_gold}
ALTER COLUMN rf_min COMMENT 'Menor valor de risco de fogo no dia'
""")

spark.sql(f"""
ALTER TABLE {catalog}.{schema_gold}.{table_gold}
ALTER COLUMN rf_max COMMENT 'Maior valor de risco de fogo no dia'
""")

spark.sql(f"""
ALTER TABLE {catalog}.{schema_gold}.{table_gold}
ALTER COLUMN qtde_pontos_grade COMMENT 'Quantidade de pixels analisados no grid para o dia'
""")

spark.sql(f"""
ALTER TABLE {catalog}.{schema_gold}.{table_gold}
ALTER COLUMN processado_em COMMENT 'Timestamp de processamento na camada Gold'
""")

print("📝 Comentários da GOLD adicionados com sucesso! ✅")
